In [361]:
#Some imports
import numpy as np
import matplotlib.pyplot as plt
import random
import queue
import time

#Functions that will be very useful for creating/updating mazes

'''
Define the grid to be working with
            **inputs**
-dim = dimension size of the grid
-p = probability that a grid spot will be filled or open
            **returns**
-maze = the grid to be worked with
'''


def grid(dim, p):
    #start with a dim by dim zero array
    maze = np.zeros((dim,dim))
    for item in range(dim):
        for thing in range(dim):
            #makes sure the top left spot is empty
            if item == 0 and thing == 0:
                pass
            #makes sure the bottom right spot is empty
            elif item == dim - 1 and thing == dim - 1:
                pass
            #change the cells based off of the value of p and our random number, x
            else:
                x = random.random()
                #if our random number is less than p, then the cell will not be filled
                if p < x:
                    maze[item][thing] = 0
                #if our random number is greater than p, then the cell will  be filled
                else:
                    maze[item][thing] = 1
    #return the grid to be worked with
    return maze


'''
update the state of the maze after moving to the next tile
            **inputs**
-maze = the maze to be updated
-i = which row to update
-j = which column to update
'''


def update(maze, i, j):
    #shades the tile grey to distinguish between open and occupied
    maze[i][j] = 0.5


'''
Euclidean Heuristic (taking the square root resulted in issues regarding rounding float values)
This was resolved by just using the sum of the squares (still gives same results)
            **inputs**
-maze = the maze being worked with
-i = the current row to use in calculation
-j = the current column to use in calculation
            **returns**
-distance = the Euclidean distance
'''


def Euclidean(maze, i, j):
    x = [len(maze) - 1, len(maze) - 1]
    y = [i, j]
    distance = np.sqrt( (y[0]-x[0])**2 + (y[1]-x[1])**2 )
    return distance


'''
Manhattan Heuristic
            **inputs**
-maze = the maze being worked with
-i = the current row to use in calculation
-j = the current column to use in calculation
            **returns**
-distance = the Manhattan distance
'''


def Manhattan(maze, i, j):
    distance = abs(len(maze)-1 - i) + abs(len(maze[0])-1 - j)
    return distance


'''
Create a grid to be worked on with an initial cell on fire
            **inputs**
-dim = the dimension size of the maze
-p = probability that a grid spot will be filled or open
            **returns**
-maze = the grid to be worked with
'''


def firegrid(dim, p):
    #start with a dim by dim zero array
    maze = np.zeros((dim,dim))
    for item in range(dim):
        for thing in range(dim):
            #makes sure the top left spot is empty
            if item == 0 and thing == 0:
                pass
            #makes sure the bottom right spot is empty
            elif item == dim - 1 and thing == dim - 1:
                pass
            #change the cells based off of the value of p and our random number
            else:
                x = random.random()
                #if our random number is less than p, then the cell will not be filled
                if p < x:
                    maze[item][thing] = 0
                #if our random number is greater than p, then the cell will  be filled
                else:
                    maze[item][thing] = 1
    #return the grid to be worked with

    r = random.randrange(0,dim-1,1)
    s = random.randrange(0,dim-1,1)
    if maze[r][s]==0:
        maze[r][s]=0.75
    else:
        while maze[r][s]==1:
            r = random.randrange(0,dim-1,1)
            s = random.randrange(0,dim-1,1)
        maze[r][s]=0.75

    return maze


'''
Count the number of neighbors on fire
            **inputs**
-maze = the maze being worked with
-i = the current row to use in calculation
-j = the current column to use in calculation
-dim = the dimension size of the maze being worked with
            **returns**
-k = the number of neighbors on fire
'''


def countFires(maze, i, j, dim):
    k=0
    if i!= dim-1 and maze[i+1][j]==0.75:
        k+=1
    if j!= dim-1 and maze[i][j+1]==0.75:
        k+=1
    if i!=0 and maze[i-1][j]==0.75:
        k+=1
    if j!=0 and maze[i][j-1]==0.75:
        k+=1
    return k


'''
Probabilistically update the fire state of every cell depending on the number of neighboring cells on fire
            **inputs**
-maze = the maze being worked with
-q = a number between 0 and 1, dictating the rate that the fire spreads
            **returns**
-maze = the updated maze
'''


def updateFire(maze, q, dim):
    for item in range(dim):
        for thing in range(dim):
            k = countFires(maze, item, thing, dim)
            #print(k)
            p = 1-((1-q)**k)
            x = random.random()
            if p > x and maze[item][thing]==0:
                maze[item][thing] = 0.75
    return maze


'''
Fire heuristic that accounts for probability of a cell being on fire 
            **inputs**
-maze = the maze being worked with
-i = the current row to use in calculation
-j = the current column to use in calculation
-q = a number between 0 and 1, dictating the rate that the fire spreads
            **returns**
-distance = the Manhattan distance with added probability a neighboring cell is going to catch on fire
'''


def firemanhattan(maze, i, j, q):
    k = countFires(maze, i, j, len(maze))
    p = 1-((1-q)**k)
    distance = abs(len(maze)-1 - i) + abs(len(maze[0])-1 - j)+(len(maze)*p)
    return distance

In [362]:
'''
BFS Search Algorithm
            **inputs**
-maze = a dim x dim array to be worked with
-video = boolean variable to either show a live update of the maze or not
-show_final = boolean variable to either display the final solution or not
            **returns**
            
-solved = 1 if solved, 0 if not
-solution_length = integer value of the final solution length
x, y = BFS(grid(100, 0.3), video = False, show_final = True) #<----- BFS Example
'''

def BFS(maze, video, show_final):
    #initialize the solved state of the maze to be false and our pointers, i and j, to be at the beginning
    #i controls row and j controls column
    solved = False
    i, j = 0, 0
    #make a copy of the maze to be used to reconstruct the final path once a solution is found
    maze_final = np.copy(maze)
    #create a dictionary called prev to point to previous positions
    prev = {}
    
    #initialize the fringe (a queue) and store the starting point of the maze
    fringe = queue.Queue()
    fringe.put((i, j))
    
    if video == True:
        plt.imshow(maze, cmap=plt.cm.binary)
        plt.pause(0.05)
    
    #runs until we reach the end
    while solved == False:
        #Is the maze unsolvable?
        if queue.Queue.qsize(fringe) == 0:
            #update the state of the maze, display the end result, and break the loop
            update(maze, i , j)
            if video == True:
                plt.imshow(maze, cmap=plt.cm.binary)
                plt.pause(0.05)
            print("UNSOLVABLE")
            #set the values of solved and solution_length to be zero
            solved = 0
            solution_length = 0
            break
        
        #gets the current node and update i and j
        current = fringe.get()
        i, j = current[0], current[1]
        
        #check if we have reached a solution, if so, display the end result and break the loop
        if i + 1 == len(maze) and j + 1 == len(maze[i]):
            update(maze, i , j)
            if video == True:
                plt.imshow(maze, cmap=plt.cm.binary)
                plt.pause(0.05)
            print("SOLUTION FOUND")
            #set the values of solved to one and initialize solution_length to be zero
            solved = 1
            solution_length = 0
    
            #start to reconstruct the path back and increment solution_length
            update(maze_final, i, j)
            if video == True:
                plt.imshow(maze, cmap=plt.cm.binary)
                plt.pause(0.05)
                
            while i != 0 or j!= 0:
                x = prev[i,j]
                i, j = x[0], x[1]
                update(maze_final, i, j)
                if video == True:
                    plt.imshow(maze_final, cmap=plt.cm.binary)
                    plt.pause(0.05)
                solution_length += 1
            
            update(maze_final, 0, 0)
            if video == True:
                plt.imshow(maze_final, cmap=plt.cm.binary)
                plt.pause(0.05)
            
            break
        
        #check down position
        
        #are we outside?
        if i + 1 >= len(maze):
            pass
        else:
            #is the next position occupied or previously visited?
            if maze[i + 1][j] == 1 or maze[i + 1][j] == 0.5:
                pass
            else:
                #add to fringe if valid and is not already in fringe
                if [i + 1, j] in fringe.queue:
                    pass
                else:
                    prev[(i + 1, j)] = (i, j)
                    fringe.put([i + 1, j])

        #check right position
        
        #are we outside?
        if j + 1 >= len(maze[i]):
            pass
        else:
            #is the next position occupied or previously visited?
            if maze[i][j + 1] == 1 or maze[i][j + 1] == 0.5:
                pass
            else:
                #add to fringe if valid and is not already in fringe
                if [i, j + 1] in fringe.queue:
                    pass
                else:
                    prev[(i, j + 1)] = (i, j)
                    fringe.put([i, j + 1])
        
        #check up solution
        
        #are we outside?
        if i - 1 < 0:
            pass
        else:
            #is the next position occupied or previously visited?
            if maze[i - 1][j] == 1 or maze[i - 1][j] == 0.5:
                pass
            else:
                #add to fringe if valid and is not already in fringe
                if [i - 1, j] in fringe.queue:
                    pass
                else:
                    prev[(i - 1, j)] = (i, j)
                    fringe.put([i - 1, j])
        
        #check left solution
        
        #are we outside?
        if j - 1 < 0:
            pass
        else:
            #is the next position occupied or previously visited?
            if maze[i][j - 1] == 1 or maze[i][j - 1] == 0.5:
                pass
            else:
                #add to fringe if valid and is not already in fringe
                if [i, j - 1] in fringe.queue:
                    pass
                else:
                    prev[(i, j - 1)] = (i, j)
                    fringe.put([i, j - 1])
        
        
        #after done checking, update the maze and keep going
        update(maze, i, j)
    
        if video == True:
            plt.imshow(maze, cmap=plt.cm.binary)
            plt.pause(0.05)
    
    if show_final == True:
           plt.figure(figsize=(10,10))
           plt.title("BFS", fontsize = 40)
           plt.imshow(maze_final, cmap=plt.cm.binary)
           plt.show()
    
    return solved, solution_length

In [363]:
'''
DFS Search Algorithm
            **inputs**
-maze = a dim x dim array to be worked with
-video = boolean variable to either show a live update of the maze or not
-show_final = boolean variable to either display the final solution or not
            **returns**
            
-solved = 1 if solved, 0 if not
-solution_length = integer value of the final solution length
-maxf = maximum size of the fringe during the search, to be used for local search
x, y, z = DFS(grid(100, 0.3), video = False, show_final = True) #<----- DFS Example
'''

def DFS(maze, video, show_final):
    ##########this is for local search only; if we enter in an unsolvable maze return 0##########
    if len(maze) == 0:
        return 0

    #initialize the solved state of the maze to be false and our pointers to be at the beginning
    #i controls row and j controls column
    solved = False
    i, j = 0, 0
    #make a copy of the maze to be used to reconstruct the final path once a solution is found
    maze_final = np.copy(maze)
    #create a dictionary called prev to point to previous positions
    prev = {}
    #largest size of fringe; initialize to 0 
    maxf = 0 
    
    #initialize the fringe (a stack) and store the starting point of the maze
    fringe = []
    fringe.append((i, j))
    
    if video == True:
        plt.imshow(maze, cmap=plt.cm.binary)
        plt.pause(0.05)
    
    #runs until we reach the end
    while solved == False:
        #update the max length of the fringe
        if len(fringe) > maxf: 
            maxf = len(fringe)
        else: 
            pass
        
        #Is the maze unsolvable?
        if len(fringe) == 0:
            #update the state of the maze, display the end result, and break the loop
            update(maze, i , j)
            if video == True:
                plt.imshow(maze, cmap=plt.cm.binary)
                plt.pause(0.05)
            print("UNSOLVABLE")
            #set the values of solved and solution_length to be zero
            solved = 0
            solution_length = 0
            break
            
        #gets the current node and update i and j
        current = fringe.pop()
        i, j = current[0], current[1]
        
        #check if we have reached a solution, if so, display the end result and break the loop
        if i + 1 == len(maze) and j + 1 == len(maze[i]):
            update(maze, i , j)
            if video == True:
                plt.imshow(maze, cmap=plt.cm.binary)
                plt.pause(0.05)
            print("SOLUTION FOUND")
            #set the values of solved to one and initialize solution_length to be zero
            solved = 1
            solution_length = 0
            
            #start to reconstruct the path back and increment solution_length
            update(maze_final, i, j)
            if video == True:
                plt.imshow(maze, cmap=plt.cm.binary)
                plt.pause(0.05)
            
            while i != 0 or j!= 0:
                x = prev[(i,j)]
                i, j = x[0], x[1]
                update(maze_final, i, j)
                if video == True:
                    plt.imshow(maze_final, cmap=plt.cm.binary)
                    plt.pause(0.05)
                solution_length += 1
            
            update(maze_final, 0, 0)
            if video == True:
                plt.imshow(maze_final, cmap=plt.cm.binary)
                plt.pause(0.05)
            
            break
            
        #check left solution
        
        #are we outside?
        if j - 1 < 0:
            pass
        else:
            #is the next position occupied or previously visited?
            if maze[i][j - 1] == 1 or maze[i][j - 1] == 0.5:
                pass
            else:
                #add to fringe if valid and is not already in fringe
                if [i, j - 1] in fringe:
                    pass
                else:
                    prev[(i, j - 1)] = (i, j)
                    fringe.append([i, j - 1])
                    
        #check up solution
        
        #are we outside?
        if i - 1 < 0:
            pass
        else:
            #is the next position occupied or previously visited?
            if maze[i - 1][j] == 1 or maze[i - 1][j] == 0.5:
                pass
            else:
                #add to fringe if valid and is not already in fringe
                if [i - 1, j] in fringe:
                    pass
                else:
                    prev[(i - 1, j)] = (i, j)
                    fringe.append([i - 1, j])
                    
        #check right position
        
        #are we outside?
        if j + 1 >= len(maze[i]):
            pass
        else:
            #is the next position occupied or previously visited?
            if maze[i][j + 1] == 1 or maze[i][j + 1] == 0.5:
                pass
            else:
                #add to fringe if valid and is not already in fringe
                if [i, j + 1] in fringe:
                    pass
                else:
                    prev[(i, j + 1)] = (i, j)
                    fringe.append([i, j + 1])
        
        #check down position
        
        #are we outside?
        if i + 1 >= len(maze):
            pass
        else:
            #is the next position occupied or previously visited?
            if maze[i + 1][j] == 1 or maze[i + 1][j] == 0.5:
                pass
            else:
                #add to fringe if valid and is not already in fringe
                if [i + 1, j] in fringe:
                    pass
                else:
                    prev[(i + 1, j)] = (i, j)
                    fringe.append([i + 1, j])        
        
        
        
        #after done checking, update the maze and start over
        update(maze, i, j)
        
        if video == True:
            plt.imshow(maze, cmap=plt.cm.binary)
            plt.pause(0.05)

    if video == True:
        plt.show()
        
    if show_final == True:
           plt.figure(figsize=(10,10))
           plt.title("DFS", fontsize = 40)
           plt.imshow(maze_final, cmap=plt.cm.binary)
           plt.show()
        
    return solved, solution_length, maxf

In [364]:
'''
BiBFS Search Algorithm
            **inputs**
-maze = a dim x dim array to be worked with
-video = boolean variable to either show a live update of the maze or not
-show_final = boolean variable to either display the final solution or not
            **returns**
            
-solved = 1 if solved, 0 if not
-solution_length = integer value of the final solution length
x, y = BiBFS(grid(100, 0.3), video = False, show_final = True) #<----- BiBFS Example
'''

def BiBFS(maze, video, show_final):
    #initialized state is set to false
    #i,j-->row,column from start m,n-->row,column from end
    solved=False
    i,j=0,0
    m,n=len(maze)-1,len(maze[0])-1
    #make a copy of the maze to be used to reconstruct the final path once a solution is found
    maze_final=np.copy(maze)
    #create two dictionaries, one to backtrack one way, one to backtrack the other way
    prev={}
    prev2={}
    
    #initialize fringe for starting point and fringe for ending point
    fringeStart=queue.Queue()
    fringeStart.put([i,j])
    fringeEnd=queue.Queue()
    fringeEnd.put([m,n])
    
    #run loop until start and end meet or no solution
    while solved==False:
        #if maze is still unsolved and there are no more children left in fringe, maze is unsolvable
        if (queue.Queue.qsize(fringeStart)==0 or queue.Queue.qsize(fringeEnd)==0):
            #update state of maze, display result, then break loop
            update(maze,i,j)
            update(maze,m,n)
            if video == True:
                plt.imshow(maze, cmap=plt.cm.binary)
                plt.pause(0.05)
            print("UNSOLVABLE")
            #set the values of solved and solution_length to be zero
            solved = 0
            solution_length = 0
            break
            
        #gets current node and updates i,j and m,n
        currentStart=fringeStart.get()
        currentEnd=fringeEnd.get()
        i,j=currentStart[0],currentStart[1]
        m,n=currentEnd[0],currentEnd[1]
        
        #check if start and end meet in middle, if so, display result and break loop
        if (i+1,j) in prev2:
            m, n = i+1, j
            solved = True
            
        if (i,j+1) in prev2:
            m, n = i, j+1
            solved = True
            
        if (i-1,j) in prev2:
            m, n = i-1, j
            solved = True
            
        if (i,j-1) in prev2:
            m,n = i, j-1
            solved = True
        
        if solved == True:
            update(maze,i,j)
            update(maze,m,n)
            if video == True:
                plt.imshow(maze, cmap=plt.cm.binary)
                plt.pause(0.05)
            print("SOLUTION FOUND")
            #set the values of solved to one and initialize solution_length
            solved = 1
            solution_length = 1
            
            #start to reconstruct the path back and increment solution_length
            update(maze_final,i,j)
            update(maze_final,m,n)
            if video == True:
                plt.imshow(maze_final, cmap=plt.cm.binary)
                plt.pause(0.05)

            while True:
                if i==0 and j==0 and m==len(maze)-1 and n==len(maze[0])-1:
                    break

                #Conditions to make sure maze reconstructs properly
                
                if (i,j) == (0,0):
                    y=prev2[(m,n)]
                    m,n=y[0],y[1]
                    update(maze_final,m,n)
                    if video == True:
                        plt.imshow(maze_final, cmap=plt.cm.binary)
                        plt.pause(0.05)
                        
                    solution_length += 1
                    
                elif (m,n) == (len(maze)-1, len(maze[0])-1):
                    x=prev[(i,j)]
                    i,j=x[0],x[1]
                    update(maze_final,i,j)
                    update(maze_final,m,n)
                    if video == True:
                        plt.imshow(maze_final, cmap=plt.cm.binary)
                        plt.pause(0.05)
                        
                    solution_length += 1
                    
                else:
                    x=prev[(i,j)]
                    y=prev2[(m,n)]
                    i,j=x[0],x[1]
                    m,n=y[0],y[1]
                    update(maze_final,i,j)
                    update(maze_final,m,n)
                    if video == True:
                        plt.imshow(maze_final, cmap=plt.cm.binary)
                        plt.pause(0.05)
                        
                    solution_length += 2
                        

            update(maze_final,0,0)
            update(maze_final,len(maze)-1,len(maze[0])-1)
            if video == True:
                plt.imshow(maze_final, cmap=plt.cm.binary)
                plt.pause(0.05)

            break
            
        #check down position of i,j
        if i + 1 >= len(maze):
            pass
        else:
            #is the next position occupied or previously visited?
            if maze[i + 1][j] == 1 or maze[i + 1][j] == 0.5:
                pass
            else:
                #add to fringe if valid and is not already in fringe
                if [i + 1, j] in fringeStart.queue:
                    pass
                else:
                    prev[(i + 1, j)] = (i, j)
                    fringeStart.put([i + 1, j])
                    
        #check up position of m,n
        if m - 1 < 0:
            pass
        else:
            #is the next position occupied or previously visited?
            if maze[m - 1][n] == 1 or maze[m - 1][n] == 0.5:
                pass
            else:
                #add to fringe if valid and is not already in fringe
                if [m - 1, n] in fringeEnd.queue:
                    pass
                else:
                    prev2[(m - 1, n)] = (m, n)
                    fringeEnd.put([m - 1, n])
                    
        #check right position of i,j
        if j + 1 >= len(maze[i]):
            pass
        else:
            #is the next position occupied or previously visited?
            if maze[i][j + 1] == 1 or maze[i][j + 1] == 0.5:
                pass
            else:
                #add to fringe if valid and is not already in fringe
                if [i, j + 1] in fringeStart.queue:
                    pass
                else:
                    prev[(i, j + 1)] = (i, j)
                    fringeStart.put([i, j + 1])
                    
        #check left position of m,n
        if n - 1 < 0:
            pass
        else:
            #is the next position occupied or previously visited?
            if maze[m][n - 1] == 1 or maze[m][n - 1] == 0.5:
                pass
            else:
                #add to fringe if valid and is not already in fringe
                if [m, n - 1] in fringeEnd.queue:
                    pass
                else:
                    prev2[(m, n - 1)] = (m, n)
                    fringeEnd.put([m, n - 1])
                    
        #check up position of i,j
        if i - 1 < 0:
            pass
        else:
            #is the next position occupied or previously visited?
            if maze[i - 1][j] == 1 or maze[i - 1][j] == 0.5:
                pass
            else:
                #add to fringe if valid and is not already in fringe
                if [i - 1, j] in fringeStart.queue:
                    pass
                else:
                    prev[(i - 1, j)] = (i, j)
                    fringeStart.put([i - 1, j])
                    
        #check down position of m,n
        if m + 1 >= len(maze):
            pass
        else:
            #is the next position occupied or previously visited?
            if maze[m + 1][n] == 1 or maze[m + 1][n] == 0.5:
                pass
            else:
                #add to fringe if valid and is not already in fringe
                if [m + 1, n] in fringeEnd.queue:
                    pass
                else:
                    prev2[(m + 1, n)] = (m, n)
                    fringeEnd.put([m + 1, n])
                    
        #check left position of i,j
        if j - 1 < 0:
            pass
        else:
            #is the next position occupied or previously visited?
            if maze[i][j - 1] == 1 or maze[i][j - 1] == 0.5:
                pass
            else:
                #add to fringe if valid and is not already in fringe
                if [i, j - 1] in fringeStart.queue:
                    pass
                else:
                    prev[(i, j - 1)] = (i, j)
                    fringeStart.put([i, j - 1])
                    
        #check right position of m,n
        if n + 1 >= len(maze[m]):
            pass
        else:
            #is the next position occupied or previously visited?
            if maze[m][n + 1] == 1 or maze[m][n + 1] == 0.5:
                pass
            else:
                #add to fringe if valid and is not already in fringe
                if [m, n + 1] in fringeEnd.queue:
                    pass
                else:
                    prev2[(m, n + 1)] = (m, n)
                    fringeEnd.put([m, n + 1])
                    
        #after done checking, update maze and keep going
        update(maze, i, j)
        update(maze,m,n)
        if video == True:
            plt.imshow(maze, cmap=plt.cm.binary)
            plt.pause(0.05)
    if video == True:        
        plt.show()
    
    if show_final == True:
           plt.figure(figsize=(10,10))
           plt.title("BiBFS", fontsize = 40)
           plt.imshow(maze_final, cmap=plt.cm.binary)
           plt.show()
    
    return solved, solution_length

In [365]:
'''
A* Euclidean Search Algorithm
            **inputs**
-maze = a dim x dim array to be worked with
-video = boolean variable to either show a live update of the maze or not
-show_final = boolean variable to either display the final solution or not
            **returns**
            
-solved = 1 if solved, 0 if not
-solution_length = integer value of the final solution length
x, y = AstarE(grid(100, 0.3), video = False, show_final = True) #<----- AstarE Example
'''

def AstarE(maze, video, show_final):
    #initialize the solved state of the maze to be false and our pointers to be at the beginning
    #i controls row and j controls column
    solved = False
    i, j = 0, 0
    #make a copy of the maze to be used to reconstruct the final path once a solution is found
    maze_final = np.copy(maze)
    #create a dictionary called prev to point to previous positions
    prev = {}
    #establish a counter to keep track of the number of moves made so far (is g(n))
    counter = 0
    
    #initialize the fringe and store the starting point of the maze
    fringe = []
    fringe.append((i, j, counter + Euclidean(maze, i, j)))
    
    if video == True:
        plt.imshow(maze, cmap=plt.cm.binary)
        plt.pause(0.05)
        
    #runs until we reach the end
    while solved == False:    
        #Is the maze unsolvable?
        if len(fringe) == 0:
            #update the state of the maze, display the end result, and break the loop
            update(maze, i , j)
            if video == True:
                plt.imshow(maze, cmap=plt.cm.binary)
                plt.show()
            print("UNSOLVABLE")
            #set the values of solved and solution_length to be zero
            solved = 0
            solution_length = 0
            break
            
        #gets the current node and update i and j
        current = fringe.pop()
        i, j = current[0], current[1]
        counter = current[2] - Euclidean(maze, i, j) + 1
        
        #check if we have reached a solution, display the end result, and break the loop
        if i + 1 == len(maze) and j + 1 == len(maze[i]):
            moves = current[2]
            update(maze, i , j)
            if video == True:
                plt.imshow(maze, cmap=plt.cm.binary)
                plt.pause(0.05)
            print("SOLUTION FOUND")
            #set the values of solved to one and initialize solution_length to be zero
            solved = 1
            solution_length = 0
            
            #start to reconstruct the path back and increment solution_length
            update(maze_final, i, j)
            if video == True:
                plt.imshow(maze, cmap=plt.cm.binary)
                plt.pause(0.05)
            
            while i != 0 or j!= 0:
                x = prev[(i,j, moves)]
                i, j, moves = x[0], x[1], x[2]
                update(maze_final, i, j)
                if video == True:
                    plt.imshow(maze_final, cmap=plt.cm.binary)
                    plt.pause(0.05)
                solution_length += 1
            
            update(maze_final, 0, 0)
            if video == True:
                plt.imshow(maze_final, cmap=plt.cm.binary)
                plt.pause(0.05)
                
            break
       
        #check left solution
        
        #are we outside?
        
        if j - 1 < 0:
            pass
        else:
            #is the next position occupied or previously visited?
            if maze[i][j - 1] == 1 or maze[i][j - 1] == 0.5:
                #If so, move on
                pass
            else:
                #check if already in fringe
                if (i, j - 1, counter + Euclidean(maze, i, j - 1)) in fringe:
                    pass
                else:
                    #if the fringe was empty (i.e. the first move), just add the first child to fringe
                    if len(fringe) == 0: 
                        prev[(i, j - 1, counter + Euclidean(maze, i, j - 1))] = (i, j, counter - 1 + Euclidean(maze, i, j))
                        fringe.append((i, j - 1, counter + Euclidean(maze, i, j - 1)))
                    #if the fringe is not empty, check the f(n) value and compare with items in fringe
                    else:
                        for x in range(len(fringe)-1,-1,-1): 
                            if counter + Euclidean(maze, i, j - 1) > fringe[x][2]:
                                if x == 0:
                                    fringe.insert(0, (i, j - 1, counter + Euclidean(maze, i, j - 1)))
                                    prev[(i, j - 1, counter + Euclidean(maze, i, j - 1))] = (i, j, counter - 1 + Euclidean(maze, i, j))
                                else:
                                    pass
                            else: 
                                fringe.insert(x + 1, (i, j - 1, counter + Euclidean(maze, i, j - 1)))
                                prev[(i, j - 1, counter + Euclidean(maze, i, j - 1))] = (i, j, counter - 1 + Euclidean(maze, i, j))
                                break 
                    
        #check up solution
        
        #are we outside?
        if i - 1 < 0:
            pass
        else:
            #is the next position occupied or previously visited?
            if maze[i - 1][j] == 1 or maze[i - 1][j] == 0.5:
                pass
            else:
                #check if already in fringe
                if (i - 1, j, counter + Euclidean(maze, i - 1, j)) in fringe:
                    pass
                else:
                    #if the fringe was empty (i.e. the first move), just add the first child to fringe
                    if len(fringe) == 0: 
                        prev[(i - 1, j, counter + Euclidean(maze, i - 1, j))] = (i, j, counter - 1 + Euclidean(maze, i, j))
                        fringe.append((i - 1, j, counter + Euclidean(maze, i - 1, j)))
                    #if the fringe is not empty, check the f(n) value and compare with items in fringe
                    else:
                        for x in range(len(fringe)-1,-1,-1): 
                            if counter + Euclidean(maze, i - 1, j) > fringe[x][2]:
                                if x == 0:
                                    fringe.insert(0, (i - 1, j, counter + Euclidean(maze, i - 1, j)))
                                    prev[(i - 1, j, counter + Euclidean(maze, i - 1, j))] = (i, j, counter - 1 + Euclidean(maze, i, j))
                                else: 
                                    pass
                            else: 
                                fringe.insert(x + 1, (i - 1, j, counter + Euclidean(maze, i - 1, j)))
                                prev[(i - 1, j, counter + Euclidean(maze, i - 1, j))] = (i, j, counter - 1 + Euclidean(maze, i, j))
                                break
                                
        #check right position
        
        #are we outside?
        if j + 1 >= len(maze[i]):
            pass
        else:
            #is the next position occupied or previously visited?
            if maze[i][j + 1] == 1 or maze[i][j + 1] == 0.5:
                pass
            else:
                #check if already in fringe
                if (i, j + 1, counter + Euclidean(maze, i, j + 1)) in fringe:
                    pass
                else:
                    #if the fringe was empty (i.e. the first move), just add the first child to fringe
                    if len(fringe) == 0: 
                        prev[(i, j + 1, counter + Euclidean(maze, i, j + 1))] = (i, j, counter - 1 + Euclidean(maze, i, j))
                        fringe.append((i, j + 1, counter + Euclidean(maze, i, j + 1)))
                    #if the fringe is not empty, check the f(n) value and compare with items in fringe
                    else:
                        for x in range(len(fringe)-1,-1,-1): 
                            if counter + Euclidean(maze, i, j + 1) > fringe[x][2]:
                                if x == 0:
                                    fringe.insert(0, (i, j + 1, counter + Euclidean(maze, i, j + 1)))
                                    prev[(i, j + 1, counter + Euclidean(maze, i, j + 1))] = (i, j, counter - 1 + Euclidean(maze, i, j))
                                else: 
                                    pass
                            else: 
                                fringe.insert(x + 1, (i, j + 1, counter + Euclidean(maze, i, j + 1)))
                                prev[(i, j + 1, counter + Euclidean(maze, i, j + 1))] = (i, j, counter - 1 + Euclidean(maze, i, j))
                                break
        
        #check down position
        
        #are we outside?
        if i + 1 >= len(maze):
            pass
        else:
            #is the next position occupied or previously visited?
            if maze[i + 1][j] == 1 or maze[i + 1][j] == 0.5:
                pass
            else:
                #check if already in fringe
                if (i + 1, j, counter + Euclidean(maze, i + 1, j)) in fringe:
                    pass
                else:
                    #if the fringe was empty (i.e. the first move), just add the first child to fringe
                    if len(fringe) == 0: 
                        prev[(i + 1, j, counter + Euclidean(maze, i + 1, j))] = (i, j, counter - 1 + Euclidean(maze, i, j))
                        fringe.append((i + 1, j, counter + Euclidean(maze, i + 1, j)))
                    #if the fringe is not empty, check the f(n) value and compare with items in fringe
                    else:
                        for x in range(len(fringe)-1,-1,-1): 
                            if counter + Euclidean(maze, i + 1, j) > fringe[x][2]:
                                if x == 0:
                                    fringe.insert(0, (i + 1, j, counter + Euclidean(maze, i + 1, j)))
                                    prev[(i + 1, j, counter + Euclidean(maze, i + 1, j))] = (i, j, counter - 1 + Euclidean(maze, i, j))
                                else: 
                                    pass
                            else: 
                                fringe.insert(x + 1, (i + 1, j, counter + Euclidean(maze, i + 1, j)))
                                prev[(i + 1, j, counter + Euclidean(maze, i + 1, j))] = (i, j, counter - 1 + Euclidean(maze, i, j))
                                break        
        
        #after done checking, update the maze and keep going
        update(maze, i, j)
        
        if video == True:
            plt.imshow(maze, cmap=plt.cm.binary)
            plt.pause(0.05)

    if show_final == True:
           plt.figure(figsize=(10,10))
           plt.title("AstarE", fontsize = 40)
           plt.imshow(maze_final, cmap=plt.cm.binary)
           plt.show()
        
    return solved, solution_length

In [366]:
'''
A* Manhattan Search Algorithm
            **inputs**
-maze = a dim x dim array to be worked with
-video = boolean variable to either show a live update of the maze or not
-show_final = boolean variable to either display the final solution or not
            **returns**
            
-solved = 1 if solved, 0 if not
-solution_length = integer value of the final solution length
-maxNode = the total number of nodes expanded, to be used for local search
x, y, z = AstarM(grid(100, 0.3), video = False, show_final = True) #<----- AstarM Example
'''

def AstarM(maze, video, show_final):
    ##########this is for local search only; if we enter in an unsolvable maze return 0##########
    if len(maze) == 0:
        return 0       
    
    #initialize the solved state of the maze to be false and our pointers to be at the beginning
    #i controls row and j controls column
    solved = False
    i, j = 0, 0
    #make a copy of the maze to be used to reconstruct the final path once a solution is found
    maze_final = np.copy(maze)
    #create a dictionary called prev to point to previous positions
    prev = {}
    #establish a counter to keep track of the number of moves made so far (is g(n))
    counter = 0
    #all nodes explored. initialize to 0 
    maxNode=0
    
    #initialize the fringe and store the starting point of the maze
    fringe = []
    fringe.append((i, j, counter + Manhattan(maze, i, j)))
    
    if video == True:
        plt.imshow(maze, cmap=plt.cm.binary)
        plt.pause(0.05)
        
    #runs until we reach the end
    while solved == False:
        #Is the maze unsolvable?
        if len(fringe) == 0:
            #update the state of the maze, display the end result, and break the loop
            update(maze, i , j)
            if video == True:
                plt.imshow(maze, cmap=plt.cm.binary)
                plt.pause(0.05)
            print("UNSOLVABLE")
            #set the values of solved and solution_length to be zero
            solved = 0
            solution_length = 0
            break
            
        #gets the current node and update i and j; update counter
        current = fringe.pop()
        #every time a node is explored, increase maxNode by 1 
        maxNode += 1
        i, j = current[0], current[1]
        counter = current[2] - Manhattan(maze, i, j) + 1
        
        #check if we have reached a solution, if so, display the end result and break the loop
        if i + 1 == len(maze) and j + 1 == len(maze[i]):
            moves = current[2]
            update(maze, i , j)
            if video == True:
                plt.imshow(maze, cmap=plt.cm.binary)
                plt.pause(0.05)
            print("SOLUTION FOUND")
            #set the values of solved to one and initialize solution_length to be zero
            solved = 1
            solution_length = 0
            
            #start to reconstruct the path back and increment solution_length
            update(maze_final, i, j)
            if video == True:
                plt.imshow(maze, cmap=plt.cm.binary)
                plt.pause(0.05)
            
            while i != 0 or j!= 0:
                x = prev[(i,j, moves)]
                i, j, moves = x[0], x[1], x[2]
                update(maze_final, i, j)
                if video == True:
                    plt.imshow(maze_final, cmap=plt.cm.binary)
                    plt.pause(0.05)
                solution_length += 1
            
            update(maze_final, 0, 0)
            if video == True:
                plt.imshow(maze_final, cmap=plt.cm.binary)
                plt.pause(0.05)
                
            break
       
        #check left solution
        
        #are we outside?
        
        if j - 1 < 0:
            pass
        else:
            #is the next position occupied or previously visited?
            if maze[i][j - 1] == 1 or maze[i][j - 1] == 0.5:
                #If so, move on
                pass
            else:
                #check if already in fringe
                if (i, j - 1, counter + Manhattan(maze, i, j - 1)) in fringe:
                    pass
                else:
                    #if the fringe was empty (i.e. the first move), just add the first child to fringe
                    if len(fringe) == 0: 
                        prev[(i, j - 1, counter + Manhattan(maze, i, j - 1))] = (i, j, counter - 1 + Manhattan(maze, i, j))
                        fringe.append((i, j - 1, counter + Manhattan(maze, i, j - 1)))
                    #if the fringe is not empty, check the f(n) value and compare with items in fringe
                    else:
                        for x in range(len(fringe)-1,-1,-1): 
                            if counter + Manhattan(maze, i, j - 1) > fringe[x][2]:
                                if x == 0:
                                    fringe.insert(0, (i, j - 1, counter + Manhattan(maze, i, j - 1)))
                                    prev[(i, j - 1, counter + Manhattan(maze, i, j - 1))] = (i, j, counter - 1 + Manhattan(maze, i, j))
                                else:
                                    pass
                            else: 
                                fringe.insert(x + 1, (i, j - 1, counter + Manhattan(maze, i, j - 1)))
                                prev[(i, j - 1, counter + Manhattan(maze, i, j - 1))] = (i, j, counter - 1 + Manhattan(maze, i, j))
                                break
        
        #check up solution
        
        #are we outside?
        if i - 1 < 0:
            pass
        else:
            #is the next position occupied or previously visited?
            if maze[i - 1][j] == 1 or maze[i - 1][j] == 0.5:
                pass
            else:
                #check if already in fringe
                if (i - 1, j, counter + Manhattan(maze, i - 1, j)) in fringe:
                    pass
                else:
                    #if the fringe was empty (i.e. the first move), just add the first child to fringe
                    if len(fringe) == 0: 
                        prev[(i - 1, j, counter + Manhattan(maze, i - 1, j))] = (i, j, counter - 1 + Manhattan(maze, i, j))
                        fringe.append((i - 1, j, counter + Manhattan(maze, i - 1, j)))
                    #if the fringe is not empty, check the f(n) value and compare with items in fringe
                    else:
                        for x in range(len(fringe)-1,-1,-1): 
                            if counter + Manhattan(maze, i - 1, j) > fringe[x][2]:
                                if x == 0:
                                    fringe.insert(0, (i - 1, j, counter + Manhattan(maze, i - 1, j)))
                                    prev[(i - 1, j, counter + Manhattan(maze, i - 1, j))] = (i, j, counter - 1 + Manhattan(maze, i, j))
                                else: 
                                    pass
                            else: 
                                fringe.insert(x + 1, (i - 1, j, counter + Manhattan(maze, i - 1, j)))
                                prev[(i - 1, j, counter + Manhattan(maze, i - 1, j))] = (i, j, counter - 1 + Manhattan(maze, i, j))
                                break
                                
        #check right position
        
        #are we outside?
        if j + 1 >= len(maze[i]):
            pass
        else:
            #is the next position occupied or previously visited?
            if maze[i][j + 1] == 1 or maze[i][j + 1] == 0.5:
                pass
            else:
                #check if already in fringe
                if (i, j + 1, counter + Manhattan(maze, i, j + 1)) in fringe:
                    pass
                else:
                    #if the fringe was empty (i.e. the first move), just add the first child to fringe
                    if len(fringe) == 0: 
                        prev[(i, j + 1, counter + Manhattan(maze, i, j + 1))] = (i, j, counter - 1 + Manhattan(maze, i, j))
                        fringe.append((i, j + 1, counter + Manhattan(maze, i, j + 1)))
                    #if the fringe is not empty, check the f(n) value and compare with items in fringe
                    else:
                        for x in range(len(fringe)-1,-1,-1): 
                            if counter + Manhattan(maze, i, j + 1) > fringe[x][2]:
                                if x == 0:
                                    fringe.insert(0, (i, j + 1, counter + Manhattan(maze, i, j + 1)))
                                    prev[(i, j + 1, counter + Manhattan(maze, i, j + 1))] = (i, j, counter - 1 + Manhattan(maze, i, j))
                                else: 
                                    pass
                            else: 
                                fringe.insert(x + 1, (i, j + 1, counter + Manhattan(maze, i, j + 1)))
                                prev[(i, j + 1, counter + Manhattan(maze, i, j + 1))] = (i, j, counter - 1 + Manhattan(maze, i, j))
                                break
                                
        #check down position
        
        #are we outside?
        if i + 1 >= len(maze):
            pass
        else:
            #is the next position occupied or previously visited?
            if maze[i + 1][j] == 1 or maze[i + 1][j] == 0.5:
                pass
            else:
                #check if already in fringe
                if (i + 1, j, counter + Manhattan(maze, i + 1, j)) in fringe:
                    pass
                else:
                    #if the fringe was empty (i.e. the first move), just add the first child to fringe
                    if len(fringe) == 0: 
                        prev[(i + 1, j, counter + Manhattan(maze, i + 1, j))] = (i, j, counter - 1 + Manhattan(maze, i, j))
                        fringe.append((i + 1, j, counter + Manhattan(maze, i + 1, j)))
                    #if the fringe is not empty, check the f(n) value and compare with items in fringe
                    else:
                        for x in range(len(fringe)-1,-1,-1): 
                            if counter + Manhattan(maze, i + 1, j) > fringe[x][2]:
                                if x == 0:
                                    fringe.insert(0, (i + 1, j, counter + Manhattan(maze, i + 1, j)))
                                    prev[(i + 1, j, counter + Manhattan(maze, i + 1, j))] = (i, j, counter - 1 + Manhattan(maze, i, j))
                                else: 
                                    pass
                            else: 
                                fringe.insert(x + 1, (i + 1, j, counter + Manhattan(maze, i + 1, j)))
                                prev[(i + 1, j, counter + Manhattan(maze, i + 1, j))] = (i, j, counter - 1 + Manhattan(maze, i, j))
                                break        
        
    
        #after done checking, update the maze and keep going
        update(maze, i, j)
        
        if video == True:
            plt.imshow(maze, cmap=plt.cm.binary)
            plt.pause(0.05)
        
    if show_final == True:
           plt.figure(figsize=(10,10))
           plt.title("AstarM", fontsize = 40)
           plt.imshow(maze_final, cmap=plt.cm.binary)
           plt.show()
        
    return solved, solution_length, maxNode

In [ ]:
#code to run final path comparisons
a = grid(75, 0.2)
b = np.copy(a)
c = np.copy(a)
d = np.copy(a)
e = np.copy(a)

BFS(a, video = False, show_final = True)
DFS(b, video = False, show_final = True)
BiBFS(c, video = False, show_final = True)
AstarM(d, video = False, show_final = True)
AstarE(e, video = False, show_final = True)

In [371]:
#HILLCLIMB WITH AstarM

#maze is filled with empty cells and obstacles
def hillClimbAstarM(maze,consec_fails):
    print('fails',consec_fails)
    #if more than 10 consecutive fails, we have reached local maxima
    if consec_fails > np.sqrt(len(maze)):
        return maze
    else:
        mazeOriginal=np.copy(maze)
        maze2 = np.copy(maze)
        #call AstarM with original maze
        algo1=AstarM(maze2,video=False, show_final = False)
        #all nodes explored with original maze is stored into maxNode1
        maxNode1=algo1[2]
        #if AstarM is solvable
        if algo1[0]==1:
            y=random.random()
            #add obstacle
            while True:
                #find a cell at i,j
                i=random.randint(1,len(maze)-2)
                j=random.randint(1,len(maze)-2)
                #if we pick a cell that already has obstacle, pass
                if maze[i][j]==1:
                    pass
                #if we pick an empty cell
                else:
                    #obstacle added in empty cell
                    maze[i][j]=1
                    mazeEdited=np.copy(maze)
                    break

            #call AstarM with new maze
            algo2=AstarM(mazeEdited,video=False, show_final = False)
            #all nodes explored of edited maze
            maxNode2=algo2[2]
            #if manxNode of edited maze is more than that of original maze
            #and AstarM of edited maze is solvable, hillClimb with edited maze
            if maxNode2>=maxNode1 and algo2[0] == 1:
                consec_fails=0
                return hillClimbAstarM(maze,consec_fails)
            else:
                #increase consecutive fails
                consec_fails+=1
                return hillClimbAstarM(mazeOriginal,consec_fails)
    print('no solution, try again (initial maze failure)')
    return []

In [372]:
#HILLCLIMB WITH DFS
#maze is filled with empty and obstacles
def hillClimbDFS(maze, consec_fails):
    print(consec_fails)
    #if there are more than 10 consistent fails, local maxima reached
    if consec_fails > np.sqrt(len(maze)):
        return maze
    else:
        mazeOriginal=np.copy(maze)
        maze2 = np.copy(maze)
        #call DFS with original maze
        algo1=DFS(maze2,video=False, show_final = False)
        fringeSize1=algo1[2]
        #if DFS is solvable
        if algo1[0]==1:
            y=random.random()
            #add obstacle
            while True:
                #find a cell at i,j
                i=random.randint(1,len(maze)-2)
                j=random.randint(1,len(maze)-2)
                #if we pick obstacle, pass
                if maze[i][j]==1:
                    pass
                #empty cell selected
                else:
                    #obstacle added in blank cell
                    maze[i][j]=1
                    mazeEdited=np.copy(maze)
                    break

            #call DFS with edited maze
            algo2=DFS(mazeEdited,video=False, show_final = False)
            #path length of edited maze
            fringeSize2=algo2[2]
            #if fringe of edited maze is bigger than fringe of original
            if fringeSize2>=fringeSize1:
                consec_fails = 0
                #continue hillClimb with edited maze as new original
                return hillClimbDFS(maze,consec_fails)
            else:
                #increase fails
                consec_fails += 1
                #continue hillClimb with original maze
                return hillClimbDFS(mazeOriginal,consec_fails)
    print('no solution, try again (initial maze failure)')
    return []

In [ ]:
'''
***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** 
This file is showing off the local search algorithms
Each hill climbing algorithm is run on the same mazes, and the before and afters are displayed
hillClimbDFS will try to maximize the total fringe size
hillCLimbAstarM will try to maximize the total nodes explored
***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** 
'''

a = grid(30, 0.2)
b = np.copy(a)
c = np.copy(a)
d = np.copy(a)

#show initial path returned by DFS
DFS(a, video = False, show_final = True)
#hill climb
harder_DFS_maze = hillClimbDFS(b, 0)
#show the final path returned by DFS
DFS(harder_DFS_maze, video = False, show_final = True)

#show initial path returned by AstarM
AstarM(c, video = False, show_final = True)
#hill climb
harder_AstarM_maze = hillClimbAstarM(d, 0)
#show the final path returned by AstarM
AstarM(harder_AstarM_maze, video = False, show_final = True)

In [368]:
'''
Maze on Fire Strategy 1
            **inputs**
-maze = a dim x dim array to be worked with
-video = boolean variable to either show a live update of the maze or not
-show_final = boolean variable to either display the final solution or not
-q = a number between 0 and 1, dictating the rate that the fire spreads
            **returns**
-solved = 1 if solved, 0 if not
-solution_length = integer value of the final solution length
x, y = fire1(firegrid(100, 0.2), video = False, show_final = True, q = 0.1) #<----- fire1 Example
'''

def fire1(maze, video, show_final, q):
    #initialize the solved state of the maze to be false and our pointers to be at the beginning
    #i controls row and j controls column
    solved = False
    onFire = False
    i, j = 0, 0
    #make a copy of the maze to be used to reconstruct the final path once a solution is found
    maze_final = np.copy(maze)
    #create a dictionary called prev to point to previous positions
    prev = {}
    #establish a counter to keep track of the number of moves made so far (is g(n))
    counter = 0

    #initialize the fringe and store the starting point of the maze
    fringe = []
    fringe.append((i, j, counter + Manhattan(maze, i, j)))

    if video == True:
        plt.imshow(maze, cmap=plt.cm.binary)
        plt.pause(0.05)

    #runs until we reach the end or we burn
    while solved == False and onFire == False:
        #check if ij is on fire
        if maze[i][j]==0.75:
            onFire=True
            print("U BURNED")
            #set the values of solved and solution_length to be zero
            solved = 0
            solution_length = 0
            break


        #Is the maze unsolvable?
        if len(fringe) == 0:
            #update the state of the maze, display the end result, and break the loop
            update(maze, i , j)
            if video == True:
                plt.imshow(maze, cmap=plt.cm.binary)
                plt.pause(0.05)
            print("UNSOLVABLE")
            #set the values of solved and solution_length to be zero
            solved = 0
            solution_length = 0
            break

        #gets the current node and update i and j; update counter
        current = fringe.pop()

        #every time a node is explored, increase maxNode by 1
        i, j = current[0], current[1]
        counter = current[2] - Manhattan(maze, i, j) + 1

        if maze[i][j]==0.75:
            onFire=True
            print("U BURNED")
            solved = 0
            solution_length = 0
            break

        #check if we have reached a solution, if so, display the end result and break the loop
        if i + 1 == len(maze) and j + 1 == len(maze[i]):
            moves = current[2]
            update(maze, i , j)
            if video == True:
                plt.imshow(maze, cmap=plt.cm.binary)
                plt.pause(0.05)
            print("SOLUTION FOUND")
            #set the values of solved to one and initialize solution_length to be zero
            solved = 1
            solution_length = 0

            #start to reconstruct the path back and increment solution_length
            update(maze_final, i, j)
            if video == True:
                plt.imshow(maze, cmap=plt.cm.binary)
                plt.pause(0.05)

            while i != 0 or j!= 0:
                x = prev[i,j, moves]
                i, j, moves = x[0], x[1], x[2]
                update(maze_final, i, j)
                if video == True:
                    plt.imshow(maze_final, cmap=plt.cm.binary)
                    plt.pause(0.05)
                solution_length += 1

            update(maze_final, 0, 0)
            if video == True:
                plt.imshow(maze_final, cmap=plt.cm.binary)
                plt.pause(0.05)

            break

        #check left solution

        #are we outside?

        if j - 1 < 0:
            pass
        else:
            #is the next position occupied or previously visited?
            if maze[i][j - 1] == 1 or maze[i][j - 1] == 0.5:
                #If so, move on
                pass
            else:
                #check if already in fringe
                if (i, j - 1, counter + Manhattan(maze, i, j - 1)) in fringe:
                    pass
                else:
                    #if the fringe was empty (i.e. the first move), just add the first child to fringe
                    if len(fringe) == 0:
                        prev[(i, j - 1, counter + Manhattan(maze, i, j - 1))] = (i, j, counter - 1 + Manhattan(maze, i, j))
                        fringe.append((i, j - 1, counter + Manhattan(maze, i, j - 1)))
                    #if the fringe is not empty, check the f(n) value and compare with items in fringe
                    else:
                        for x in range(len(fringe)-1,-1,-1):
                            if counter + Manhattan(maze, i, j - 1) > fringe[x][2]:
                                if x == 0:
                                    fringe.insert(0, (i, j - 1, counter + Manhattan(maze, i, j - 1)))
                                    prev[(i, j - 1, counter + Manhattan(maze, i, j - 1))] = (i, j, counter - 1 + Manhattan(maze, i, j))
                                else:
                                    pass
                            else:
                                fringe.insert(x + 1, (i, j - 1, counter + Manhattan(maze, i, j - 1)))
                                prev[(i, j - 1, counter + Manhattan(maze, i, j - 1))] = (i, j, counter - 1 + Manhattan(maze, i, j))
                                break

        #check up solution

        #are we outside?
        if i - 1 < 0:
            pass
        else:
            #is the next position occupied or previously visited?
            if maze[i - 1][j] == 1 or maze[i - 1][j] == 0.5:
                pass
            else:
                #check if already in fringe
                if (i - 1, j, counter + Manhattan(maze, i - 1, j)) in fringe:
                    pass
                else:
                    #if the fringe was empty (i.e. the first move), just add the first child to fringe
                    if len(fringe) == 0:
                        prev[(i - 1, j, counter + Manhattan(maze, i - 1, j))] = (i, j, counter - 1 + Manhattan(maze, i, j))
                        fringe.append((i - 1, j, counter + Manhattan(maze, i - 1, j)))
                    #if the fringe is not empty, check the f(n) value and compare with items in fringe
                    else:
                        for x in range(len(fringe)-1,-1,-1):
                            if counter + Manhattan(maze, i - 1, j) > fringe[x][2]:
                                if x == 0:
                                    fringe.insert(0, (i - 1, j, counter + Manhattan(maze, i - 1, j)))
                                    prev[(i - 1, j, counter + Manhattan(maze, i - 1, j))] = (i, j, counter - 1 + Manhattan(maze, i, j))
                                else:
                                    pass
                            else:
                                fringe.insert(x + 1, (i - 1, j, counter + Manhattan(maze, i - 1, j)))
                                prev[(i - 1, j, counter + Manhattan(maze, i - 1, j))] = (i, j, counter - 1 + Manhattan(maze, i, j))
                                break

        #check right position

        #are we outside?
        if j + 1 >= len(maze[i]):
            pass
        else:
            #is the next position occupied or previously visited?
            if maze[i][j + 1] == 1 or maze[i][j + 1] == 0.5:
                pass
            else:
                #check if already in fringe
                if (i, j + 1, counter + Manhattan(maze, i, j + 1)) in fringe:
                    pass
                else:
                    #if the fringe was empty (i.e. the first move), just add the first child to fringe
                    if len(fringe) == 0:
                        prev[(i, j + 1, counter + Manhattan(maze, i, j + 1))] = (i, j, counter - 1 + Manhattan(maze, i, j))
                        fringe.append((i, j + 1, counter + Manhattan(maze, i, j + 1)))
                    #if the fringe is not empty, check the f(n) value and compare with items in fringe
                    else:
                        for x in range(len(fringe)-1,-1,-1):
                            if counter + Manhattan(maze, i, j + 1) > fringe[x][2]:
                                if x == 0:
                                    fringe.insert(0, (i, j + 1, counter + Manhattan(maze, i, j + 1)))
                                    prev[(i, j + 1, counter + Manhattan(maze, i, j + 1))] = (i, j, counter - 1 + Manhattan(maze, i, j))
                                else:
                                    pass
                            else:
                                fringe.insert(x + 1, (i, j + 1, counter + Manhattan(maze, i, j + 1)))
                                prev[(i, j + 1, counter + Manhattan(maze, i, j + 1))] = (i, j, counter - 1 + Manhattan(maze, i, j))
                                break

        #check down position

        #are we outside?
        if i + 1 >= len(maze):
            pass
        else:
            #is the next position occupied or previously visited?
            if maze[i + 1][j] == 1 or maze[i + 1][j] == 0.5:
                pass
            else:
                #check if already in fringe
                if (i + 1, j, counter + Manhattan(maze, i + 1, j)) in fringe:
                    pass
                else:
                    #if the fringe was empty (i.e. the first move), just add the first child to fringe
                    if len(fringe) == 0:
                        prev[(i + 1, j, counter + Manhattan(maze, i + 1, j))] = (i, j, counter - 1 + Manhattan(maze, i, j))
                        fringe.append((i + 1, j, counter + Manhattan(maze, i + 1, j)))
                    #if the fringe is not empty, check the f(n) value and compare with items in fringe
                    else:
                        for x in range(len(fringe)-1,-1,-1):
                            if counter + Manhattan(maze, i + 1, j) > fringe[x][2]:
                                if x == 0:
                                    fringe.insert(0, (i + 1, j, counter + Manhattan(maze, i + 1, j)))
                                    prev[(i + 1, j, counter + Manhattan(maze, i + 1, j))] = (i, j, counter - 1 + Manhattan(maze, i, j))
                                else:
                                    pass
                            else:
                                fringe.insert(x + 1, (i + 1, j, counter + Manhattan(maze, i + 1, j)))
                                prev[(i + 1, j, counter + Manhattan(maze, i + 1, j))] = (i, j, counter - 1 + Manhattan(maze, i, j))
                                break


        #after done checking, update the maze and keep going
        updateFire(maze, q, len(maze))
        update(maze, i, j)


        if video == True:
            plt.imshow(maze, cmap=plt.cm.binary)
            plt.pause(0.05)

    if show_final == True:
           plt.figure(figsize=(10,10))
           plt.title("AstarM", fontsize = 40)
           plt.imshow(maze_final, cmap=plt.cm.binary)
           plt.show()

    return solved, solution_length

In [369]:
'''
Maze on Fire Strategy 2
            **inputs**
-maze = a dim x dim array to be worked with
-video = boolean variable to either show a live update of the maze or not
-show_final = boolean variable to either display the final solution or not
-q = a number between 0 and 1, dictating the rate that the fire spreads
            **returns**
-solved = 1 if solved, 0 if not
-solution_length = integer value of the final solution length
x, y = fire2(firegrid(50, 0.2), video = True, show_final = False, q = 0.1) #<----- fire2 Example
'''

def fire2(maze, video, show_final, q):
    #initialize the solved state of the maze to be false and our pointers to be at the beginning
    #i controls row and j controls column
    solved = False
    onFire = False
    i, j = 0, 0
    #make a copy of the maze to be used to reconstruct the final path once a solution is found
    maze_final = np.copy(maze)
    #create a dictionary called prev to point to previous positions
    prev = {}
    #establish a counter to keep track of the number of moves made so far (is g(n))
    counter = 0

    #initialize the fringe and store the starting point of the maze
    fringe = []
    fringe.append((i, j, counter + Manhattan(maze, i, j)))

    if video == True:
        plt.imshow(maze, cmap=plt.cm.binary)
        plt.pause(0.05)

    #runs until we reach the end or burn
    while solved == False and onFire == False:
        #check if ij is on fire
        #if maze[i][j]==0.75:
        #    onFire=True
        #    print("U BURNED")
        #    #set the values of solved and solution_length to be zero
        #    solved = 0
        #    solution_length = 0
        #    break


        #Is the maze unsolvable?
        if len(fringe) == 0:
            #update the state of the maze, display the end result, and break the loop
            update(maze, i , j)
            if video == True:
                plt.imshow(maze, cmap=plt.cm.binary)
                plt.pause(0.05)
            print("UNSOLVABLE")
            #set the values of solved and solution_length to be zero
            solved = 0
            solution_length = 0
            break

        #gets the current node and update i and j; update counter
        current = fringe.pop()

        i, j = current[0], current[1]
        counter = current[2] - Manhattan(maze, i, j) + 1

        #if the cell we are looking at is filled with fire, dont go there!
        if maze[i][j] == 0.75:
            continue

        #check if we have reached a solution, if so, display the end result and break the loop
        if i + 1 == len(maze) and j + 1 == len(maze[i]):
            moves = current[2]
            update(maze, i , j)
            if video == True:
                plt.imshow(maze, cmap=plt.cm.binary)
                plt.pause(0.05)
            print("SOLUTION FOUND")
            #set the values of solved to one and initialize solution_length to be zero
            solved = 1
            solution_length = 0

            #start to reconstruct the path back and increment solution_length
            update(maze_final, i, j)
            if video == True:
                plt.imshow(maze, cmap=plt.cm.binary)
                plt.pause(0.05)

            while i != 0 or j!= 0:
                x = prev[i,j, moves]
                i, j, moves = x[0], x[1], x[2]
                update(maze_final, i, j)
                if video == True:
                    plt.imshow(maze_final, cmap=plt.cm.binary)
                    plt.pause(0.05)
                solution_length += 1

            update(maze_final, 0, 0)
            if video == True:
                plt.imshow(maze_final, cmap=plt.cm.binary)
                plt.pause(0.05)

            break

        #check left solution

        #are we outside?

        if j - 1 < 0:
            pass
        else:
            #is the next position occupied or previously visited?
            if maze[i][j - 1] == 1 or maze[i][j - 1] == 0.5 or maze[i][j - 1] == 0.75:
                #If so, move on
                pass
            else:
                #check if already in fringe
                if (i, j - 1, counter + Manhattan(maze, i, j - 1)) in fringe:
                    pass
                else:
                    #if the fringe was empty (i.e. the first move), just add the first child to fringe
                    if len(fringe) == 0:
                        prev[(i, j - 1, counter + Manhattan(maze, i, j - 1))] = (i, j, counter - 1 + Manhattan(maze, i, j))
                        fringe.append((i, j - 1, counter + Manhattan(maze, i, j - 1)))
                    #if the fringe is not empty, check the f(n) value and compare with items in fringe
                    else:
                        for x in range(len(fringe)-1,-1,-1):
                            if counter + Manhattan(maze, i, j - 1) > fringe[x][2]:
                                if x == 0:
                                    fringe.insert(0, (i, j - 1, counter + Manhattan(maze, i, j - 1)))
                                    prev[(i, j - 1, counter + Manhattan(maze, i, j - 1))] = (i, j, counter - 1 + Manhattan(maze, i, j))
                                else:
                                    pass
                            else:
                                fringe.insert(x + 1, (i, j - 1, counter + Manhattan(maze, i, j - 1)))
                                prev[(i, j - 1, counter + Manhattan(maze, i, j - 1))] = (i, j, counter - 1 + Manhattan(maze, i, j))
                                break

        #check up solution

        #are we outside?
        if i - 1 < 0:
            pass
        else:
            #is the next position occupied or previously visited?
            if maze[i - 1][j] == 1 or maze[i - 1][j] == 0.5 or maze[i - 1][j] == 0.75:
                pass
            else:
                #check if already in fringe
                if (i - 1, j, counter + Manhattan(maze, i - 1, j)) in fringe:
                    pass
                else:
                    #if the fringe was empty (i.e. the first move), just add the first child to fringe
                    if len(fringe) == 0:
                        prev[(i - 1, j, counter + Manhattan(maze, i - 1, j))] = (i, j, counter - 1 + Manhattan(maze, i, j))
                        fringe.append((i - 1, j, counter + Manhattan(maze, i - 1, j)))
                    #if the fringe is not empty, check the f(n) value and compare with items in fringe
                    else:
                        for x in range(len(fringe)-1,-1,-1):
                            if counter + Manhattan(maze, i - 1, j) > fringe[x][2]:
                                if x == 0:
                                    fringe.insert(0, (i - 1, j, counter + Manhattan(maze, i - 1, j)))
                                    prev[(i - 1, j, counter + Manhattan(maze, i - 1, j))] = (i, j, counter - 1 + Manhattan(maze, i, j))
                                else:
                                    pass
                            else:
                                fringe.insert(x + 1, (i - 1, j, counter + Manhattan(maze, i - 1, j)))
                                prev[(i - 1, j, counter + Manhattan(maze, i - 1, j))] = (i, j, counter - 1 + Manhattan(maze, i, j))
                                break

        #check right position

        #are we outside?
        if j + 1 >= len(maze[i]):
            pass
        else:
            #is the next position occupied or previously visited?
            if maze[i][j + 1] == 1 or maze[i][j + 1] == 0.5 or maze[i][j + 1] == 0.75:
                pass
            else:
                #check if already in fringe
                if (i, j + 1, counter + Manhattan(maze, i, j + 1)) in fringe:
                    pass
                else:
                    #if the fringe was empty (i.e. the first move), just add the first child to fringe
                    if len(fringe) == 0:
                        prev[(i, j + 1, counter + Manhattan(maze, i, j + 1))] = (i, j, counter - 1 + Manhattan(maze, i, j))
                        fringe.append((i, j + 1, counter + Manhattan(maze, i, j + 1)))
                    #if the fringe is not empty, check the f(n) value and compare with items in fringe
                    else:
                        for x in range(len(fringe)-1,-1,-1):
                            if counter + Manhattan(maze, i, j + 1) > fringe[x][2]:
                                if x == 0:
                                    fringe.insert(0, (i, j + 1, counter + Manhattan(maze, i, j + 1)))
                                    prev[(i, j + 1, counter + Manhattan(maze, i, j + 1))] = (i, j, counter - 1 + Manhattan(maze, i, j))
                                else:
                                    pass
                            else:
                                fringe.insert(x + 1, (i, j + 1, counter + Manhattan(maze, i, j + 1)))
                                prev[(i, j + 1, counter + Manhattan(maze, i, j + 1))] = (i, j, counter - 1 + Manhattan(maze, i, j))
                                break

        #check down position

        #are we outside?
        if i + 1 >= len(maze):
            pass
        else:
            #is the next position occupied or previously visited?
            if maze[i + 1][j] == 1 or maze[i + 1][j] == 0.5 or maze[i + 1][j] == 0.75:
                pass
            else:
                #check if already in fringe
                if (i + 1, j, counter + Manhattan(maze, i + 1, j)) in fringe:
                    pass
                else:
                    #if the fringe was empty (i.e. the first move), just add the first child to fringe
                    if len(fringe) == 0:
                        prev[(i + 1, j, counter + Manhattan(maze, i + 1, j))] = (i, j, counter - 1 + Manhattan(maze, i, j))
                        fringe.append((i + 1, j, counter + Manhattan(maze, i + 1, j)))
                    #if the fringe is not empty, check the f(n) value and compare with items in fringe
                    else:
                        for x in range(len(fringe)-1,-1,-1):
                            if counter + Manhattan(maze, i + 1, j) > fringe[x][2]:
                                if x == 0:
                                    fringe.insert(0, (i + 1, j, counter + Manhattan(maze, i + 1, j)))
                                    prev[(i + 1, j, counter + Manhattan(maze, i + 1, j))] = (i, j, counter - 1 + Manhattan(maze, i, j))
                                else:
                                    pass
                            else:
                                fringe.insert(x + 1, (i + 1, j, counter + Manhattan(maze, i + 1, j)))
                                prev[(i + 1, j, counter + Manhattan(maze, i + 1, j))] = (i, j, counter - 1 + Manhattan(maze, i, j))
                                break


        #after done checking, update the maze and keep going
        updateFire(maze, q, len(maze))
        update(maze, i, j)


        if video == True:
            plt.imshow(maze, cmap=plt.cm.binary)
            plt.pause(0.05)

    if show_final == True:
           plt.figure(figsize=(7,7))
           plt.title("AstarM", fontsize = 40)
           plt.imshow(maze_final, cmap=plt.cm.binary)
           plt.show()

    return solved, solution_length

In [370]:
'''
Maze on Fire Strategy 3
            **inputs**
-maze = a dim x dim array to be worked with
-video = boolean variable to either show a live update of the maze or not
-show_final = boolean variable to either display the final solution or not
-q = a number between 0 and 1, dictating the rate that the fire spreads
            **returns**
-solved = 1 if solved, 0 if not
-solution_length = integer value of the final solution length
x, y = fire3(firegrid(50, 0.2), video = True, show_final = False, q = 0.1) #<----- fire3 Example
'''


def fire3(maze, video, show_final, q):
    #initialize the solved state of the maze to be false and our pointers to be at the beginning
    #i controls row and j controls column
    solved = False
    onFire = False
    i, j = 0, 0
    #make a copy of the maze to be used to reconstruct the final path once a solution is found
    maze_final = np.copy(maze)
    #create a dictionary called prev to point to previous positions
    prev = {}
    #establish a counter to keep track of the number of moves made so far (is g(n))
    counter = 0
    #all nodes explored. initialize to 0
    maxNode=0

    #initialize the fringe and store the starting point of the maze
    fringe = []
    fringe.append((i, j, counter + firemanhattan(maze, i, j, q)))

    if video == True:
        plt.imshow(maze, cmap=plt.cm.binary)
        plt.pause(0.05)

    #runs until we reach the end
    while solved == False and onFire == False:
        #check if ij is on fire
        #if maze[i][j]==0.75:
        #    onFire=True
        #    print("U BURNED")
        #    solved = 0
        #    solution_length = 0
        #    break


        #Is the maze unsolvable?
        if len(fringe) == 0:
            #update the state of the maze, display the end result, and break the loop
            update(maze, i , j)
            if video == True:
                plt.imshow(maze, cmap=plt.cm.binary)
                plt.pause(0.05)
            print("UNSOLVABLE")
            #set the values of solved and solution_length to be zero
            solved = 0
            solution_length = 0
            break

        #gets the current node and update i and j; update counter
        current = fringe.pop()

        #every time a node is explored, increase maxNode by 1
        maxNode += 1
        i, j = current[0], current[1]
        counter = current[2] - firemanhattan(maze, i, j, q) + 1

        #if the cell we are looking at is filled with fire, dont go there!
        if maze[i][j] == 0.75:
            continue

        #check if we have reached a solution, if so, display the end result and break the loop
        if i + 1 == len(maze) and j + 1 == len(maze[i]):
            moves = current[2]
            update(maze, i , j)
            if video == True:
                plt.imshow(maze, cmap=plt.cm.binary)
                plt.pause(0.05)
            print("SOLUTION FOUND")
            #set the values of solved to one and initialize solution_length to be zero
            solved = 1
            solution_length = 0

            #start to reconstruct the path back and increment solution_length
            update(maze_final, i, j)
            if video == True:
                plt.imshow(maze, cmap=plt.cm.binary)
                plt.pause(0.05)

            while i != 0 or j!= 0:
                x = prev[i,j, moves]
                i, j, moves = x[0], x[1], x[2]
                update(maze_final, i, j)
                if video == True:
                    plt.imshow(maze_final, cmap=plt.cm.binary)
                    plt.pause(0.05)
                solution_length += 1

            update(maze_final, 0, 0)
            if video == True:
                plt.imshow(maze_final, cmap=plt.cm.binary)
                plt.pause(0.05)

            break

        #check left solution

        #are we outside?

        if j - 1 < 0:
            pass
        else:
            #is the next position occupied or previously visited?
            if maze[i][j - 1] == 1 or maze[i][j - 1] == 0.5 or maze[i][j - 1] == 0.75:
                #If so, move on
                pass
            else:
                #check if already in fringe
                if (i, j - 1, counter + firemanhattan(maze, i, j - 1,q)) in fringe:
                    pass
                else:
                    #if the fringe was empty (i.e. the first move), just add the first child to fringe
                    if len(fringe) == 0:
                        prev[(i, j - 1, counter + firemanhattan(maze, i, j - 1,q))] = (i, j, counter - 1 + firemanhattan(maze, i, j,q))
                        fringe.append((i, j - 1, counter + firemanhattan(maze, i, j - 1,q)))
                    #if the fringe is not empty, check the f(n) value and compare with items in fringe
                    else:

                        for x in range(len(fringe)-1,-1,-1):
                            if counter + firemanhattan(maze, i, j - 1,q) > fringe[x][2]:
                                if x == 0:
                                    fringe.insert(0, (i, j - 1, counter + firemanhattan(maze, i, j - 1,q)))
                                    prev[(i, j - 1, counter + firemanhattan(maze, i, j - 1,q))] = (i, j, counter - 1 + firemanhattan(maze, i, j,q))
                                else:
                                    pass
                            else:
                                fringe.insert(x + 1, (i, j - 1, counter + firemanhattan(maze, i, j - 1,q)))
                                prev[(i, j - 1, counter + firemanhattan(maze, i, j - 1,q))] = (i, j, counter - 1 + firemanhattan(maze, i, j,q))
                                break

        #check up solution

        #are we outside?
        if i - 1 < 0:
            pass
        else:
            #is the next position occupied or previously visited?
            if maze[i - 1][j] == 1 or maze[i - 1][j] == 0.5 or maze[i - 1][j] == 0.75:
                pass
            else:
                #check if already in fringe
                if (i - 1, j, counter + firemanhattan(maze, i - 1, j,q)) in fringe:
                    pass
                else:
                    #if the fringe was empty (i.e. the first move), just add the first child to fringe
                    if len(fringe) == 0:
                        prev[(i - 1, j, counter + firemanhattan(maze, i - 1, j,q))] = (i, j, counter - 1 + firemanhattan(maze, i, j,q))
                        fringe.append((i - 1, j, counter + firemanhattan(maze, i - 1, j, q)))
                    #if the fringe is not empty, check the f(n) value and compare with items in fringe
                    else:
                        for x in range(len(fringe)-1,-1,-1):
                            if counter + firemanhattan(maze, i - 1, j, q) > fringe[x][2]:
                                if x == 0:
                                    fringe.insert(0, (i - 1, j, counter + firemanhattan(maze, i - 1, j, q)))
                                    prev[(i - 1, j, counter + firemanhattan(maze, i - 1, j, q))] = (i, j, counter - 1 + firemanhattan(maze, i, j, q))
                                else:
                                    pass
                            else:
                                fringe.insert(x + 1, (i - 1, j, counter + firemanhattan(maze, i - 1, j, q)))
                                prev[(i - 1, j, counter + firemanhattan(maze, i - 1, j, q))] = (i, j, counter - 1 + firemanhattan(maze, i, j, q))
                                break

        #check right position

        #are we outside?
        if j + 1 >= len(maze[i]):
            pass
        else:
            #is the next position occupied or previously visited?
            if maze[i][j + 1] == 1 or maze[i][j + 1] == 0.5 or maze[i][j + 1] == 0.75:
                pass
            else:
                #check if already in fringe
                if (i, j + 1, counter + firemanhattan(maze, i, j + 1, q)) in fringe:
                    pass
                else:
                    #if the fringe was empty (i.e. the first move), just add the first child to fringe
                    if len(fringe) == 0:
                        prev[(i, j + 1, counter + firemanhattan(maze, i, j + 1, q))] = (i, j, counter - 1 + firemanhattan(maze, i, j, q))
                        fringe.append((i, j + 1, counter + firemanhattan(maze, i, j + 1, q)))
                    #if the fringe is not empty, check the f(n) value and compare with items in fringe
                    else:
                        for x in range(len(fringe)-1,-1,-1):
                            if counter + firemanhattan(maze, i, j + 1, q) > fringe[x][2]:
                                if x == 0:
                                    fringe.insert(0, (i, j + 1, counter + firemanhattan(maze, i, j + 1, q)))
                                    prev[(i, j + 1, counter + firemanhattan(maze, i, j + 1, q))] = (i, j, counter - 1 + firemanhattan(maze, i, j, q))
                                else:
                                    pass
                            else:
                                fringe.insert(x + 1, (i, j + 1, counter + firemanhattan(maze, i, j + 1,q)))
                                prev[(i, j + 1, counter + firemanhattan(maze, i, j + 1, q))] = (i, j, counter - 1 + firemanhattan(maze, i, j, q))
                                break

        #check down position

        #are we outside?
        if i + 1 >= len(maze):
            pass
        else:
            #is the next position occupied or previously visited?
            if maze[i + 1][j] == 1 or maze[i + 1][j] == 0.5 or maze[i + 1][j] == 0.75:
                pass
            else:
                #check if already in fringe
                if (i + 1, j, counter + firemanhattan(maze, i + 1, j, q)) in fringe:
                    pass
                else:
                    #if the fringe was empty (i.e. the first move), just add the first child to fringe
                    if len(fringe) == 0:
                        prev[(i + 1, j, counter + firemanhattan(maze, i + 1, j, q))] = (i, j, counter - 1 + firemanhattan(maze, i, j, q))
                        fringe.append((i + 1, j, counter + firemanhattan(maze, i + 1, j, q)))
                    #if the fringe is not empty, check the f(n) value and compare with items in fringe
                    else:
                        for x in range(len(fringe)-1,-1,-1):
                            if counter + firemanhattan(maze, i + 1, j, q) > fringe[x][2]:
                                if x == 0:
                                    fringe.insert(0, (i + 1, j, counter + firemanhattan(maze, i + 1, j, q)))
                                    prev[(i + 1, j, counter + firemanhattan(maze, i + 1, j, q))] = (i, j, counter - 1 + firemanhattan(maze, i, j, q))
                                else:
                                    pass
                            else:
                                fringe.insert(x + 1, (i + 1, j, counter + firemanhattan(maze, i + 1, j, q)))
                                prev[(i + 1, j, counter + firemanhattan(maze, i + 1, j, q))] = (i, j, counter - 1 + firemanhattan(maze, i, j, q))
                                break


        #after done checking, update the maze and keep going
        updateFire(maze, q, len(maze))
        update(maze, i, j)


        if video == True:
            plt.imshow(maze, cmap=plt.cm.binary)
            plt.pause(0.05)

    if show_final == True:
           plt.figure(figsize=(7,7))
           plt.title("AstarM", fontsize = 40)
           plt.imshow(maze_final, cmap=plt.cm.binary)
           plt.show()

    return solved, solution_length, maxNode

In [ ]:
'''
***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** 
For running the different strategies on the same mazes 
-fire will still spread differently for each maze, but starting position is the same
***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** 
'''

#make copies of the same maze to be run by each algorithm
a = firegrid(15, 0.2)
b = np.copy(a)
c = np.copy(a)

print("Strategy 1: Regular A*Manhattan")
time.sleep(2)
fire1(a, video = True, show_final = False, q = 0.1)
print()
print("Strategy 2: Adapt to Environment")
time.sleep(2)
fire2(b, video = True, show_final = False, q = 0.1)
print()
print("Strategy 3: Plan Ahead")
time.sleep(2)
fire3(c, video = True, show_final = False, q = 0.1)